
**[PT]** Português

---

**[EN]** English


Antes da primeira execução instalar os requisitos segundos as instruções em [README.md](./README.md)

---

Before first run install the requirements following the instructions in [README.md](./README.md)



# Visão geral da base de dados

---

# Database overview 


## Setup

In [1]:
from timelink.mhk.utilities import get_dbnames

db_names = get_dbnames()
print(db_names)


['china_coimbra', 'mhk', 'soure_ipm', 'tests', 'toliveira', 'toliveira_china', 'ucalumni', 'ucprosop']


In [2]:
from timelink.mhk.utilities import get_connection_string
from timelink.mhk.models.db import TimelinkMHK 
import timelink.notebooks.config as conf

db_name = 'tests'

db_conn_string = get_connection_string(db_name)
db = TimelinkMHK(db_conn_string)
# store it in the config for later use
conf.TIMELINK_DBSYSTEM = db

In [3]:
conf.TIMELINK_DBSYSTEM

## Database status

In [3]:
from sqlalchemy.sql import func
from timelink.mhk.models import Entity


with db.session() as session:
    classes = session.query(Entity.pom_class,func.count(Entity.pom_class)).group_by(Entity.pom_class).all()
    for c,n in classes:
        print(f'{n:8} | {c}')

    2117 | act
      28 | acta
     126 | aforamento
   24892 | attribute
       5 | carta
      45 | cartaperdao
      28 | class
      45 | crime
      52 | divida
     223 | escritura
      42 | evento
      27 | geoentity
     112 | good
    1972 | group_element
     628 | household
       4 | item
      91 | object
   13053 | person
   24733 | relation
     166 | rperson
       6 | siza
      39 | source


## Atributos

---

## Atributes

In [4]:
from sqlalchemy import func
from sqlalchemy import select, desc
from timelink.mhk.models import Attribute


attr_table = Attribute.__table__

stmt = select(attr_table.c.the_type,func.count().label('tot')).group_by('the_type').order_by(desc('tot'))
print(stmt)
print()
with db.session() as session:
    nml = session.execute(stmt)
    for the_type, tot in nml:
        print(f'{tot:6} | {the_type}')


SELECT attributes.the_type, count(*) AS tot 
FROM attributes GROUP BY attributes.the_type ORDER BY tot DESC

  4129 | ec
  2603 | residencia
  1281 | naturalidade
  1119 | titulo
  1022 | morto
   913 | matricula
   913 | matricula.ano
   910 | cargo
   812 | profissao
   660 | morada
   593 | posicao
   592 | local_registro
   570 | nome.apelido
   558 | freguesia
   523 | assina
   474 | produto_fornecido
   406 | viaja
   295 | nome-geografico
   291 | nome
   288 | código-de-referência
   288 | data-do-registo
   288 | url
   288 | uc.entrada
   288 | uc.saida
   288 | nome.primeiro
   275 | nome-geografico.ano
   267 | naturalidade.ano
   267 | uc.entrada.ano
   267 | uc.saida.ano
   258 | idade_aprox
   257 | faculdade
   257 | faculdade.ano
   230 | grau
   230 | grau.ano
   225 | datanasc
   190 | exame
   154 | localizacao
   138 | foro
   125 | instituta
   125 | instituta.ano
   114 | assinatura
    93 | morta
    82 | alcunha
    80 | racao
    57 | patrimonio
    54 | valo

## Exemplo de interface entre Timelink e Pandas

---

## Examples of Timelink/Pandas interface

Pesquisa por nomes

---

Search by names


In [5]:
from timelink.pandas import pname_to_df


df = pname_to_df('Joaquim Lopes', similar=True)
df.reset_index(inplace=True)
df

,index,id,name,sex,obs
0,0,r1776-f60-joaquim,joaquim lopes,m,None
1,1,r1777-f64-per190,joaquim lopes,m,None
2,2,b1760.539-per7,joaquim vimieiro lopes,m,None
3,3,b1762.589-per8,joaquim vimieiro lopes,m,None


Listar informações de um nome

---

List information about a name

In [11]:
from sqlalchemy import select
from timelink.mhk.models import Person

for id in df.id:
    with db.session() as session:
        person = session.get(Person, id)
        print(person.to_kleio())
        print()

filho$joaquim lopes/m/id=r1776-f60-joaquim
  rel$function-in-act/filho/joaquim lopes/r1776/17760511

n$joaquim lopes/m/id=r1777-f64-per190
  pai$bento lopes/m/id=r1777-f64-per190-per191/obs=ja nao vive no fogo
    rel$function-in-act/pai/bento lopes/r1777/17770505
  mulher$luisa mauricia/f/id=r1777-f64-per190-per192
    rel$function-in-act/mulher/luisa mauricia/r1777/17770505
  rel$function-in-act/n/joaquim lopes/r1777/17770505

pad$joaquim vimieiro lopes/m/id=b1760.539-per7
  rel$function-in-act/pad/joaquim vimieiro lopes/b1760.539/17600512
  ls$titulo/reverendo doutor/17600512
  ls$cargo/vigario de brasfemes/17600512

pad$joaquim vimieiro lopes/m/id=b1762.589-per8
  rel$function-in-act/pad/joaquim vimieiro lopes/b1762.589/17620816



### Exemplo de registo complexo

---

### Example of complex record

Example of the current capabilities of the algorithm including correction of “Faculdade”, religious order, titles, enrollment at class level, exam results and degrees.

In [6]:
from timelinknb.pandas import group_attributes
from ucalumni.extractors import get_extractors
from ucalumni.aluno import get_and_process_aluno

id = '316297'

exts = get_extractors()
print("Extractors currently active:")
[print("  ",ex.__name__) for ex in exts]
print()

print()
print("Copy the information to a markdown file it should render nicely")
print("------------------------")
aluno = get_and_process_aluno(id,db)
print(aluno)
print("------------------------")


ModuleNotFoundError: No module named 'timelinknb'

#### Mostrar os attributos como uma tabela

---

#### Show the attributes as a table

In [6]:

pd.set_option('display.max_rows',1000)
pd.set_option('display.max_colwidth',100)

df = group_attributes([id])
df = df[~df['type'].str.contains(".", regex = False)]  # remove redundant attributes with "." in the name
df = df[['date','type','value','attr_obs']].sort_values(['date','type','value'])
df.fillna(" ")

,date,type,value,attr_obs
id,,,,
316297,1774-11-15,faculdade,Cânones,Faculdade corrigida
316297,1774-11-15,faculdade-original,Direito - Cânones,
316297,1774-11-15,matricula-classe,"Curso jurídico, 1º ano","Vol. III, L. I, fl. 20v."
316297,1774-11-15,matricula-curso,Curso jurídico,"Vol. III, L. I, fl. 20v."
316297,1774-11-15,naturalidade,Porto,
316297,1774-11-15,nome,João Pedro Ribeiro,
316297,1774-11-15,nome-apelido,Pedro Ribeiro,
316297,1774-11-15,nome-apelido,Ribeiro,
316297,1774-11-15,nome-geografico,Porto,
